# Determining 1-skeleton for potential $k$-simplices using adjacency matrix of a graph $G$

   *Tyler Foster*

   *Abdullah Malik*

## Introduction
Let $A=A^{1}$ be the adjacency matrix associated with the graph $G_{\bullet }
$ with $\left\vert G_{1}\right\vert =g$ vertices. By that, we mean that the
entry $a_{ij}^{\left( 1\right) }=1$ if there is edge from vertex $i$ to
vertex $j$, and zero otherwise. 

To apply the machinary of simplicial sets in Machine Learning, we first need to upscale our data from a graph $G_{\bullet }$ to its $\text{conn}^k(G_\bullet)$. This requires identification of vertices for its 1-skeleton $v_{0},v_{1},...,v_{k}$. 

Consider $A^{k}$, with entries $a_{ij}^{\left( k\right) }$. If $%
a_{ij}^{\left( k\right) }=\ell \not=0$, then there are $\ell $ paths of
length $k$ from $v_{i}=v_{\beta _{1}}\longrightarrow v_{\beta
_{2}}\longrightarrow ...\longrightarrow v_{\beta _{k}}=v_{j}$, for each $%
\beta \in \left[ \ell \right] ^{\times }$, where $\left[ \ell \right]
^{\times }=\left\{ 1,2,...,\ell \right\} $. Hence, there is a potential $%
\left( k-1\right) $-simplex. The existence of such a potential simplex is
discredited immediately if $a_{ij}^{\left( k-1\right) }=0$.

The idea, therefore, is to collect $k-1$ out-neighbours of vertex $i$ and $k-1$ in-neighbours of vertex $j$, provided that $a_{ij}^{\left( k-1\right)
}\not=0$, to determine $1$-skeleton of a potential $\left( k-1\right) $-simplex.

The supporting theory is pushed in the appendix

## Code
We initialize our graph as a DGL, heterograph. As an example, we use the graph with 8 vertices and 2 3-simplices, connected by a 1-simplex.

In [ ]:
import dgl
import numpy as np
import itertools

One of the inputs requires setting the size the highest dimension $d$ to be searched for. If, however, there are no potential $k$-simplices to be found, for $k<d$, the algorithm automatically terminates.

In [ ]:
class Adj_Graph_idx_Paths():
    src=list()
    dst=list()
    empty_graph = dgl.heterograph({('node', 'to', 'node'): (src, dst)})
    upper_dimension = int
    
    assert isinstance(empty_graph, dgl.DGLHeteroGraph), \
        'Keyword argument \"graph\" of AdjGraph\'s init methodmust be a dgl.DGLHeteroGraph.'
    
    def __init__(
        self, graph=empty_graph, 
        dimension=upper_dimension):
        self.seed_graph = graph
        src, dst = self.seed_graph.edges()
        pair_edge = tensor(zip(src, dst))
        src = src.tolist()
        dst = dst.tolist()
        self.dimension = dimension
        set_of_nodes = set(src+dst)
        seed_nodes = list(set_of_nodes)
        self.seed_nodes=seed_nodes
        
        """
        These dictionaries are keyed with a dimension, and the corresponding values are vertices the
        graph should be searched for to determine potential simplices. Because our graphs are directed, 
        we will need two dictionaries, one for each direction. Both are then combined, on the same principle,
        in the subgraph_dictionary.
        """
        
        self.simplex_dictionary  = dict()
        self.potential_simplices = dict()
        
        for i in range(dimension+1):
            self.simplex_dictionary[i]  = list()
            self.potential_simplices[i] = list()
            
        self.simplex_dictionary[0]=self.seed_nodes

        """Most of the calculations below follow rely on the manipulation of an adjacency matrix of the 
        graph, given as a coordinate matrix A=adj_matrix_coo. Powers of each adjacency matrix are stored in
        another list.
        """
        
        A=self.seed_graph.adj(scipy_fmt='coo')
        self.adj_matrix_coo=A
        B=list()
        B.append(list())
        B[0]=A
        # A list with k-th entry corresponding to the power A^{k+1}
        self.powers_of_adj=B
        
    def TwoSimplicesConstruction(self):
        self.powers_of_adj.append(self.powers_of_adj[0].dot(self.adj_matrix_coo))
        for row_next, col_next in zip(*self.powers_of_adj[1].nonzero()):
            # Temporary sets to capture vertices. Sets are opted to avoid redundancies in final collection.
            outlist=list()
            inlist=list()
            if self.powers_of_adj[0].tocsr()[row_next,col_next] != 0:
                for row_prev, col_prev in zip(*self.powers_of_adj[0].nonzero()):
                    if row_prev == row_next:
                        outlist.append(col_prev)
                    if col_prev == col_next:
                        inlist.append(row_prev)
                for vertex in list(set(outlist).intersection(inlist)):
                    temp = self.simplex_dictionary[2]
                    temp = temp + [[row_next] + [vertex] + [col_next]]
                    self.simplex_dictionary[2] = temp
        
    def iterating_through(self):
        # Run through all powers of A^k
        for k in range(2,self.dimension+1):
            # This part over here will cause a slow-down, as we're multiplying matrices in their usual fashion.
            self.powers_of_adj.append(self.powers_of_adj[k-2].dot(self.adj_matrix_coo))
            # Record coordinates of the coordinate matrix corresponding to A^{k+1}
            row_indices, column_indices = self.powers_of_adj[k-1].nonzero()
            # If there is no vertex from which originates a path of length k, then we need to stop our search
            if len(row_indices) == 0:
                return k
    
            """The first thing we do is look for a pair of vertices, corresponding to entries in A^{k+1}, labelled
            row_next and col_next. Once these are found, then we look at the same entry for A^k. If this same entry
            is also nonzero, then we record this pair.
            """
        
            for nrow, ncol in zip(*self.powers_of_adj[k-1].nonzero()):
            # Temporary sets to capture vertices. Sets are opted to avoid redundancies in final collection.
                out_simplices=list()
                in_simplices=list()
                out_neighbors=list()
                in_neighbors=list()
                if self.powers_of_adj[k-2].tocsr()[nrow,ncol] != 0:
                    for row_prev, col_prev in zip(*self.powers_of_adj[0].nonzero()):
                        if row_prev == nrow:
                            outlist.append(col_prev)
                        if col_prev == ncol:
                            inlist.append(row_prev)
                    #for vertex in list(set(outlist).intersection(inlist)):
                        
                    temp = self.potential_simplices[k]
                    temp = temp + list(set(outlist).intersection(inlist)) + [nrow] + [ncol]
                    self.potential_simplices[k] = temp

The next function is built to make maximal simplices

In [ ]:
class SimplexCreator():
    
    def __init__(self, dimension):
        self.input_dimension = dimension
        self.src=list()
        self.dst=list()
        for i in range(self.input_dimension+1):
            for j in range(self.input_dimension+1):
                if (i < j):
                    self.src = self.src + [i]
                    self.dst = self.dst + [j]

In [ ]:
""" Code testing"""

K_3 = dgl.heterograph({('paper', 'cites', 'paper'): (SimplexCreator(dimension=3).src, SimplexCreator(dimension=3).dst)})

K_3_preprocessing = Adj_Graph_idx_Paths(graph=K_3,dimension=10)
K_3_preprocessing.TwoSimplicesConstruction()
K_3_preprocessing.iterating_through()
print(K_3_preprocessing.simplex_dictionary)
print(K_3_preprocessing.potential_simplices)

In [ ]:
""" Code testing"""
src_simplex = [0,0,0,1,1,2] + [4,4,4,5,5,6] + [1]
dst_simplex = [1,2,3,2,3,3] + [5,6,7,6,7,7] + [4]
two_simplexes = dgl.heterograph({('paper', 'cites', 'paper'): (src_simplex, dst_simplex)})

two_simplxes_preprocessing = Adj_Graph_idx_Paths(graph=two_simplexes,dimension=20)
two_simplxes_preprocessing.TwoSimplicesConstruction()
print(two_simplxes_preprocessing.simplex_dictionary)
print(two_simplxes_preprocessing.potential_simplices)

In [ ]:
""" Code testing"""
K_5 = dgl.heterograph({('paper', 'cites', 'paper'): (SimplexCreator(dimension=5).src, SimplexCreator(dimension=5).dst)})

K_5_preprocessing = Adj_Graph_idx_Paths(graph=K_5,dimension=10)
K_5_preprocessing.TwoSimplicesConstruction()
K_5_preprocessing.iterating_through()
print(K_5_preprocessing.simplex_dictionary)
print(K_5_preprocessing.potential_simplices)

In [ ]:
""" Code testing"""
K_7 = dgl.heterograph({('paper', 'cites', 'paper'): (SimplexCreator(dimension=7).src, SimplexCreator(dimension=7).dst)})

K_7_preprocessing = Adj_Graph_idx_Paths(graph=K_7,dimension=10)
K_7_preprocessing.iterating_through()
print(K_7_preprocessing.in_neighbours_dictionary)
print(K_7_preprocessing.out_neighbours_dictionary)
print(K_7_preprocessing.subgraph_dictionary)

## Appendix
Let $A=A^{1}$ be the adjacency matrix associated with the graph $G_{\bullet
} $ with $\left\vert G_{1}\right\vert =g$ vertices. By that, we mean that
the entry $a_{ij}^{\left( 1\right) }=1$ if there is edge from vertex $v_{i}$
to vertex $v_{j}$, and zero otherwise. Consider $A^{2}$, with entry 
\begin{equation*}
a_{ij}^{\left( 2\right) }=\overset{g}{\underset{k=1}{\sum }}a_{ik}^{\left(
1\right) }a_{kj}^{\left( 1\right) }.
\end{equation*}
For any $k$, the product $a_{ik}^{\left( 1\right) }a_{kj}^{\left( 1\right) }$
is $1$ if there is path from vertex $v_{i}$ to vertex $v_{k}$, and from
vertex $v_{k}$ to vertex $v_{j}$. Thus, $a_{ij}^{\left( 2\right) }$ is
corresponds to the number of paths of length 2 from vertex $v_{i}$ to vertex 
$v_{j}$. In addition, if $a_{ij}^{\left( 1\right) }=1$, then we know for
sure that not only is there the path $v_{i}\longrightarrow
v_{k}\longrightarrow v_{j}$, but also $v_{i}\longrightarrow v_{j}$.

We need to identify vertices $v_{0},v_{1},...,v_{k}$ corresponding to a $k$
-simplex. The $0$-simplices are simply the vertices, whereas to gather $1$
-simplices, we simply ''compute'' $A^{1}=A$
. The set of $1$-simplices is such that $S_{1}=\left\{ \left[ v_{i},v_{j}%
\right] :a_{ij}^{\left( 1\right) }=1\right\} $.

Now, for a given $i,j$, if $a_{ij}^{\left( 2\right) }=\ell \not=0$, then $%
\exists k_{1},k_{2}...,k_{\ell }\in \mathbb{N}$ such that $a_{ik_{\alpha
}}^{\left( 1\right) }=a_{k_{\alpha }j}^{\left( 1\right) }=1$, where $1\leq
\alpha \leq \ell $, and $1\leq k_{\alpha }\leq g$. Thus, the set of
potential $2$-simplices is $S_{2}=\left\{ \left[ v_{i},v_{k_{\alpha }},v_{j}%
\right] :\left( a_{ij}^{\left( 2\right) }=\ell \not=0\right) \wedge \forall
\alpha \left( a_{ik_{\alpha }}^{\left( 1\right) }=a_{k_{\alpha }j}^{\left(
1\right) }=1\right) \right\} $. Note that $\ell $ depends on $i$ and $j$.
The only hurdle is to show that $a_{ij}^{\left( 1\right) }=1$. Therefore, to
find $2$ simplices in a graph using its adjacency matrix, we simply look at
nonzero entries $a_{ij}^{\left( 2\right) }$ and scan through the
out-nieghbors ouf vertex $i$, and in-nieghbours of vertex $j$. That is, row $%
i$ and column $j$.

To look for $3$-simplices, observe that 
\begin{eqnarray*}
a_{ij}^{\left( 3\right) } &=&\overset{g}{\underset{k=1}{\sum }}%
a_{ik}^{\left( 1\right) }a_{kj}^{\left( 2\right) }=\overset{g}{\underset{k=1}%
{\sum }}a_{ik}^{\left( 1\right) }\left( \overset{g}{\underset{m=1}{\sum }}%
a_{km}^{\left( 1\right) }a_{mj}^{\left( 1\right) }\right) \\
&=&\overset{g}{\underset{k=1}{\sum }}\overset{g}{\underset{m=1}{\sum }}%
a_{ik}^{\left( 1\right) }a_{km}^{\left( 1\right) }a_{mj}^{\left( 1\right) }
\end{eqnarray*}
Thus, $a_{ij}^{\left( 3\right) }=\ell \not=0$ tells us that $\exists
k_{1},k_{2}...,k_{\ell }$ and $m_{1},m_{2}...,m_{\ell }\in \mathbb{N}$ such
that $a_{ik_{\beta }}^{\left( 1\right) }=a_{k_{\beta }m_{\beta }}^{\left(
1\right) }=a_{m_{\beta }j}^{\left( 1\right) }=1$, and the potential, $\ell $ 
$3$-simplices are $\left[ v_{i},v_{k_{\beta }},v_{m_{\beta }},v_{j}\right] $%
, where $1\leq \beta \leq \ell $.

#### Lemma
Let $\left[ v_{i},v_{k_{\beta }},v_{m_{\beta }},v_{j}\right] \in S_{3}$,
where $1\leq \beta \leq \ell $, and $a_{ij}^{\left( 3\right) }=\ell $. Then, 
$\forall \beta $, $\exists \alpha $ such that $\left( v_{i},v_{\alpha
},v_{j}\right) \in S_{2}$.


#### Proof
Let $\beta \in \left\{ 1,2,...,\ell \right\} $ be a fixed number. Then
\begin{eqnarray*}
1 &=&\overset{g}{\underset{k_{\beta }=1}{\sum }}\overset{g}{\underset{%
m_{\beta }=1}{\sum }}a_{ik_{\beta }}^{\left( 1\right) }a_{k_{\beta
}m_{\beta }}^{\left( 1\right) }a_{m_{\beta }j}^{\left( 1\right) }=\overset{g}%
{\underset{k_{\beta }=1}{\sum }}a_{ik_{\beta }}^{\left( 1\right) }\left(
\overset{g}{\underset{m_{\beta }=1}{\sum }}a_{k_{\beta }m_{\beta }}^{\left(
1\right) }a_{m_{\beta }j}^{\left( 1\right) }\right)  \\
&=&\overset{g}{\underset{k_{\beta }=1}{\sum }}a_{ik_{\beta }}^{\left(
1\right) }a_{k_{\beta }j}^{\left( 2\right) }=\overset{g}{\underset{m_{\beta
}=1}{\sum }}a_{m_{\beta }j}^{\left( 1\right) }\left( \overset{g}{\underset{%
k_{\beta }=1}{\sum }}a_{ik_{\beta }}^{\left( 1\right) }a_{k_{\beta
}m_{\beta }}^{\left( 1\right) }\right) =\overset{g}{\underset{m_{\beta }=1}{%
\sum }}a_{m_{\beta }j}^{\left( 1\right) }a_{im_{\beta }}^{\left( 2\right) }
\end{eqnarray*}
This sum can only be 1 if $\exists \alpha $ such that $a_{\alpha j}^{\left(
2\right) }=1$, or if $\exists \alpha $ such that $a_{i\alpha }^{\left(
2\right) }=1$. In other words, $k_{\beta }=\alpha $, or $m_{\beta }=\alpha $.∎


Therefore, to look for $3$-simplices, one looks for nonzero entries $%
a_{ij}^{\left( 3\right) }$, nonzero entries in the $i$-th row of $A^{2}$ to
determine $\alpha $'s as above, and the $j$-th columns of $A^{2}$.

In general, $a_{ij}^{\left( n\right) }$ corresponds to the number of paths
of length $n$ from vertex $i$ to vertex $j$, and to find $n$-simplices in
the graph, the equality 
\begin{equation*}
a_{ij}^{\left( n\right) }=\overset{g}{\underset{\beta _{1}=1}{\sum }}\overset%
{g}{\underset{\beta _{2}=1}{\sum }}...\overset{g}{\underset{\beta _{n-1}=1}{%
\sum }}a_{i\beta _{1}}^{\left( 1\right) }a_{\beta _{1}\beta _{2}}^{\left(
1\right) }...a_{\beta _{n-2}\beta _{n-1}}^{\left( 1\right) }a_{\beta
_{n-1}j}^{\left( 1\right) }=\ell
\end{equation*}
tells us that there exists integers $\beta _{1,1},\beta _{1,2},...,\beta
_{1,\ell },\beta _{2,1},\beta _{2,2},...,\beta _{2,\ell },....,\beta
_{n-1,\ell }$ between $0$ and $g$, such that $\left[ v_{i},v_{\beta _{1,\mu
}},v_{\beta _{2,\mu }},...,v_{\beta _{n-1,\mu }},v_{j}\right] $ are the
vertices for the $\mu $-th $n$-simplex, where $1\leq \mu \leq \ell $.
Moreover, for each of the $\ell $ $n$-simplices, $\exists \alpha $ such that 
$a_{i\alpha }^{\left( n-1\right) }=1$ or $a_{\alpha j}^{\left( n-1\right)
}=1 $.